In [46]:
import logging

In [47]:
logging.basicConfig( 
filename='etl_log.txt', 
level=logging.INFO, 
format='%(asctime)s - %(levelname)s - %(message)s' 
) 

In [48]:
import pandas as pd

In [49]:
import numpy as np

In [50]:
import mysql.connector

In [51]:
import datetime

In [52]:
csv_file_path = 'employees1.csv'

In [53]:
df = pd.read_csv("C:/Users/sabar/Downloads/employees1.csv")


In [54]:
print("Raw data loaded:") 

Raw data loaded:


In [55]:
print(df.head())

   EMPLOYEE_ID FIRST_NAME  LAST_NAME     EMAIL  PHONE_NUMBER  HIRE_DATE  \
0          198     Donald   OConnell  DOCONNEL  650.507.9833  21-Jun-07   
1          199    Douglas      Grant    DGRANT  650.507.9844  13-Jan-08   
2          200   Jennifer     Whalen   JWHALEN  515.123.4444  17-Sep-03   
3          201    Michael  Hartstein  MHARTSTE  515.123.5555  17-Feb-04   
4          202        Pat        Fay      PFAY  603.123.6666  17-Aug-05   

     JOB_ID  SALARY  
0  SH_CLERK    2600  
1  SH_CLERK    2600  
2   AD_ASST    4400  
3    MK_MAN   13000  
4    MK_REP    6000  


In [56]:
print(df.columns.tolist())


['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE', 'JOB_ID', 'SALARY']


In [57]:
logging.info("CSV loaded successfully.")

In [58]:
df.fillna({ 
'EMAIL': 'not_provided@example.com', 
'PHONE_NUMBER': '0000000000', 
'HIRE_DATE': '01-Jan-00', 
'SALARY': 0 
}, inplace=True) 


In [59]:
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

In [60]:
print(df.columns.tolist())

['employee_id', 'first_name', 'last_name', 'email', 'phone_number', 'hire_date', 'job_id', 'salary']


In [61]:
df['hire_date'] = pd.to_datetime(df["hire_date"], format='%d-%b-%y', 
errors='coerce')

In [62]:
df['hire_date'] = df['hire_date'].fillna(pd.to_datetime('2000-01-01')) 

In [63]:
df['salary'] = pd.to_numeric(df['salary'], errors='coerce').fillna(0).astype(int)


In [64]:
logging.info("Data cleaning completed.")


In [66]:
mydb = mysql.connector.connect( 
    host="localhost", 
    user="root", 
    password="Parthalachu2015", 
    database="empolyee"  
) 
cursor = mydb.cursor() 

In [67]:
 
cursor.execute(""" 
    CREATE TABLE IF NOT EXISTS salary_2 ( 
        empid INT, 
        firstname VARCHAR(50), 
        lastname VARCHAR(50), 
        email VARCHAR(100), 
        phone VARCHAR(20), 
        hire_date DATE, 
        job_id VARCHAR(20), 
        salary INT 
    ) 
""") 

In [70]:
for index, row in df.iterrows(): 
    sql = """ 
      INSERT INTO salary_2 (empid, firstname, lastname, email, phone, 
hire_date, job_id, salary) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
            firstname=VALUES(firstname), 
            lastname=VALUES(lastname), 
            email=VALUES(email), 
            phone=VALUES(phone), 
            hire_date=VALUES(hire_date), 
            job_id=VALUES(job_id), 
            salary=VALUES(salary) 
    """ 
    values = ( 
        int(row['employee_id']),
           row['first_name'], 
        row['last_name'], 
        row['email'], 
        row['phone_number'], 
        row['hire_date'].date(), 
        row['job_id'], 
        int(row['salary']) 
    ) 
    cursor.execute(sql, values)
    

In [71]:
mydb.commit()

In [72]:
cursor.close()

True

In [73]:
mydb.close() 

In [74]:
logging.error("Something went wrong", exc_info=True)

In [75]:
print("ETL process completed successfully.")

ETL process completed successfully.


In [1]:
import os 
print(os.getcwd()) 
get_ipython().system('jupyter nbconvert --to script ETL.ipynb')

C:\Users\sabar


[NbConvertApp] Converting notebook ETL.ipynb to script
[NbConvertApp] Writing 2863 bytes to ETL.py
